# Transfer Learning Tutorial

Sources:

- [cs231n @ Stanford](https://cs231n.github.io/transfer-learning/)

- [Sasank Chilamkurthy](https://chsasank.github.io)


Quote [cs231n @ Stanford](https://cs231n.github.io/transfer-learning/):

In practice, very few people train an entire Convolutional Network
from scratch (with random initialization), because it is relatively
rare to have a dataset of sufficient size. Instead, it is common to
pretrain a ConvNet on a very large dataset (e.g. ImageNet, which
contains 1.2 million images with 1000 categories), and then use the
ConvNet either as an initialization or a fixed feature extractor for
the task of interest.

These two major transfer learning scenarios look as follows:

-  **ConvNet as fixed feature extractor**:
    * Take a ConvNet pretrained on ImageNet,
    * Remove the last fully-connected layer (this layer’s outputs are the 1000 class scores for a different task like ImageNet)
    * Treat the rest of the ConvNet as a fixed feature extractor for the new dataset.

    In practice:
    * Freeze the weights for all of the network except that of the final fully connected layer. This last fully connected layer is replaced with a new one with random weights and only this layer is trained.

-  **Finetuning the convnet**: 

fine-tune the weights of the pretrained network by continuing the backpropagation. It is possible to fine-tune all the layers of the ConvNet

Instead of random initializaion, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.

In [ ]:
%matplotlib inline

## Train and test functions

In [ ]:
import os
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

n_epochs = 5
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
random_seed = 1

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


def train(model, criterion, train_loader, optimizer, epoch, device, log_interval=None,
          batch_max=np.inf, save_model=True):
    train_losses, train_counter = list(), list()
    # epoch = 1; log_interval=10; train_losses=[]; train_counter=[]

    model.train()

    # Iterate over minibatch
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx > batch_max:
            break
        # batch_idx, (data, target) = next(enumerate(train_loader))
        # print(data.shape)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
    
        # Forward
        output = model(data)
        loss = criterion(output, target)
    
        # Bakward
        loss.backward()

        # Update params
        optimizer.step()
        
        # Track losses
        train_losses.append(loss.item())
        train_counter.append(data.shape[0]) # (batch_idx * data.shape[0]) + ((epoch-1)*len(train_loader.dataset)))

        # Save model
        if (log_interval is not None) and batch_idx % log_interval == 0:
            print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    return model, train_losses, train_counter


def test(model, criterion, test_loader, device, batch_max=np.inf):

    model.eval()

    test_loss = 0
    correct = 0
    output, pred, target = list(), list(), list()

    # Iterate over mini-batches
    with torch.no_grad():
        for batch_idx, (data, target_) in enumerate(test_loader):
            if batch_idx > batch_max:
                break
            # batch_idx, (data, target) = next(enumerate(test_loader))
            # print(target_.shape)
            data, target_ = data.to(device), target_.to(device) # target.shape == 1000
            output_ = model(data) # output.shape == (1000, 10)
            
            # Compute loss
            running_loss += loss.item() * inputs.size(0)

            test_loss += criterion(output_, target_, reduction='sum').item() # sum up batch loss
            pred_ = output_.argmax(dim=1) # get the index of the max log-probability
            
            # An correct classification
            correct += pred_.eq(target_.view_as(pred_)).sum().item() # view_as(other): View this tensor as the same size as other

            # Track output, class-prediction and true target
            output.append(output_)
            pred.append(pred_)
            target.append(target_)

    output = torch.cat(output)
    pred = torch.cat(pred)
    target = torch.cat(target)
    assert pred.eq(target.view_as(pred)).sum().item() == correct

    test_loss /= len(target)
    print('Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)'.format(
        test_loss, correct, len(target),
        100. * correct / len(target)))
    return pred, output, target, test_loss


def epochs(model, criterion, optimizer, train_loader, test_loader, device, log_interval=None,
          batch_max=np.inf, save_model=False):

    train_losses, train_counter, test_losses, test_counter = [], [], [], []
    for epoch in range(1, n_epochs + 1):
        print("=== Epoch %i ===" % epoch)
        model, train_losses_, train_counter_ = train(model, criterion, train_loader, optimizer, epoch,
                                                     device, log_interval=log_interval, batch_max=batch_max)

        train_losses += train_losses_
        train_counter += train_counter_

        # Train accuracy
        print("Train: ", end = '')
        pred_train, output_train, target_train, loss_train = test(model, criterion, train_loader, device)
        #print("Train accuracy = {:.1f}%".format((target_train == pred_train).sum().item() * 100. / len(target_train)))
        #print("Test accuracy = {:.1f}%".format((target == pred).sum().item() * 100. / len(target)))

        print("Test : ", end = '')
        pred, output, target, test_loss = test(model, criterion, test_loader, device)
        test_counter.append(np.sum(train_counter))
        test_losses.append(test_loss)


        if save_model:
            torch.save(model.state_dict(), 'models/mod-%s.pth' % model.__class__.__name__)
            torch.save(optimizer.state_dict(), 'models/mod-%s_opt-%s.pth' % (model.__class__.__name__, optimizer.__class__.__name__))

    fig = plt.figure()
    plt.plot(np.cumsum(train_counter), train_losses, color='blue')
    plt.plot(test_counter, test_losses, "or")
    plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
    plt.xlabel('number of training examples seen')
    plt.ylabel('negative log likelihood loss')

## CIFAR-10 dataset

[Source Yunjey Choi](https://github.com/yunjey/pytorch-tutorial)

In [ ]:
from pathlib import Path
WD = os.path.join(Path.home(), "data", "pystatml", "dl_cifar10_pytorch")
os.makedirs(WD, exist_ok=True)
os.chdir(WD)
print("Working dir is:", os.getcwd())
os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 5
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

# Info about the dataset
data_shape = train_loader.dataset.data.shape[1:]
D_in = np.prod(data_shape)
D_out = len(set(train_loader.dataset.targets))
print(data_shape, D_in, D_out)

### ResNet as a feature extractor

Freeze all the network except the final layer: `requires_grad == False` to freeze the parameters so that the gradients are not computed in backward()

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 10)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

In [ ]:
n_epochs, model = 5,  model_conv.to(device) # 195738 parameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Explore the model
#for parameter in model.parameters():
#    print(parameter.shape)
print("Total number of parameters =", np.sum([np.prod(parameter.shape) for parameter in model.parameters()]))

epochs(model, criterion, optimizer, train_loader, test_loader, device)


In [ ]:
test(model, criterion, test_loader, device, batch_max=np.inf)
# _, preds = torch.max(outputs, 1)


In [ ]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

import os, sys
import tempfile

In [ ]:
WD = os.path.join(tempfile.gettempdir(), "dl_transfer-learning_ants-bees_pytorch")
os.makedirs(WD, exist_ok=True)
os.chdir(WD)
print("Working dir is:", os.getcwd())
os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)

In [ ]:
!ls
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip --output-document=data/hymenoptera_data.zip
!unzip data/hymenoptera_data.zip -d data/

## Load Data


We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

Note: Download the data from `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>` and extract it to the current directory.

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Visualize a few images

Let's visualize a few training images so as to understand the data
augmentations.

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

print(inputs.shape)

imshow(out, title=[class_names[x] for x in classes])

## Training the model

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

## Visualizing the model predictions


Generic function to display predictions for a few images

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

## Finetuning the convnet

Load a pretrained model and reset final fully connected layer.

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### Train and evaluate

It should take around 15-25 min on CPU. On GPU though, it takes less than a
minute.

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [ ]:
visualize_model(model_ft)

## ConvNet as fixed feature extractor

Here, we need to freeze all the network except the final layer. We need
to set ``requires_grad == False`` to freeze the parameters so that the
gradients are not computed in ``backward()``.

You can read more about this in the documentation
`here <https://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward>`.


In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

### Train and evaluate

On CPU this will take about half the time compared to previous scenario.
This is expected as gradients don't need to be computed for most of the
network. However, forward does need to be computed.

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

In [ ]:
visualize_model(model_conv)

plt.ioff()
plt.show()